In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
import matplotlib.pyplot as plt

In [3]:
path = './input/'

In [4]:
train = pd.read_csv(path+'pfm_train.csv')
train.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,37,0,Travel_Rarely,Research & Development,1,4,Life Sciences,77,1,Male,2,2,Manufacturing Director,3,Divorced,5993,1,Y,No,18,3,3,80,1,7,2,4,7,5,0,7
1,54,0,Travel_Frequently,Research & Development,1,4,Life Sciences,1245,4,Female,3,3,Manufacturing Director,3,Divorced,10502,7,Y,No,17,3,1,80,1,33,2,1,5,4,1,4
2,34,1,Travel_Frequently,Research & Development,7,3,Life Sciences,147,1,Male,1,2,Laboratory Technician,3,Single,6074,1,Y,Yes,24,4,4,80,0,9,3,3,9,7,0,6
3,39,0,Travel_Rarely,Research & Development,1,1,Life Sciences,1026,4,Female,2,4,Manufacturing Director,4,Married,12742,1,Y,No,16,3,3,80,1,21,3,3,21,6,11,8
4,28,1,Travel_Frequently,Research & Development,1,3,Medical,1111,1,Male,2,1,Laboratory Technician,2,Divorced,2596,1,Y,No,15,3,1,80,2,1,2,3,1,0,0,0


In [5]:
len(train)

1100

In [6]:
test = pd.read_csv(path+'pfm_test.csv')
# test.head()

In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [8]:
test['Attrition'] = -1

In [9]:
data = train
data = data.append(test).reset_index(drop=True)

C:\Users\wang.kui\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [10]:
len(data)

1450

In [11]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['Attrition']]
for i in cat_col:
#     data[i] = OneHotEncoder().fit_transform(data[i])
    data[i] = LabelEncoder().fit_transform(data[i])

In [12]:
feats = [i  for i  in data.columns if i not in ['Attrition']]
# feats

In [13]:
train_x = data[data['Attrition'] != -1][feats]
train_y = data[data['Attrition'] != -1]['Attrition']
test_x = data[data['Attrition'] == -1][feats]

In [14]:
train_x.shape, train_y.shape, test_x.shape

((1100, 30), (1100,), (350, 30))

In [15]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

In [16]:
model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=70, reg_alpha=0., reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary', metric='F1',
    subsample=0.95, colsample_bytree=0.95, subsample_frew=1, 
    learning_rate=0.002, random_state=2017)
model.fit(train_x, train_y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.95,
        importance_type='split', learning_rate=0.002, max_depth=-1,
        metric='F1', min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=1500, n_jobs=-1, num_leaves=70,
        objective='binary', random_state=2017, reg_alpha=0.0,
        reg_lambda=0.0, silent=True, subsample=0.95,
        subsample_for_bin=200000, subsample_freq=0, subsample_frew=1)

In [18]:
from sklearn.model_selection import KFold
n_splits=5
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [32]:
from sklearn import metrics
def measures(test_y, pred):    
    res = [[]]
    res[0].append(metrics.accuracy_score(test_y, pred))
    res[0].append(metrics.precision_score(test_y, pred))
    res[0].append(metrics.recall_score(test_y, pred))
    res[0].append(metrics.f1_score(test_y, pred))
    res[0].append(metrics.roc_auc_score(test_y, pred))
    
    print("accuracy      : %0.3f" % res[0][0])
    print("precision score: %0.3f" % res[0][1])
    print("recall score  : %0.3f" % res[0][2])
    print("f1 score      : %0.3f" % res[0][3])
    print("roc-auc score      : %0.3f" % res[0][4])
    data = pd.DataFrame(res, columns=['accuracy', 'precision', 'recall', 'f1', 'auc'])
    return data

mea_res=pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1', 'auc'])
# mea_res

In [42]:
model = lgb.LGBMClassifier(boosting_type='gbdt', reg_alpha=0., reg_lambda=0.,learning_rate=0.3, random_state=2017)

train_x = data[data['Attrition'] != -1][feats]
train_y = data[data['Attrition'] != -1]['Attrition']
test_x = data[data['Attrition'] == -1][feats]
res = data[data['Attrition'] == -1][['Attrition']]

res['Attrition']=0
for train_idx, val_idx in kfold.split(train_x):
    model.random_state = model.random_state+1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x.loc[val_idx]
    test_y1 = train_y.loc[val_idx]
    model.fit(train_x1, train_y1, eval_set=[(train_x1, train_y1), (test_x1, test_y1)],
        eval_metric='auc', early_stopping_rounds=100)
    res['Attrition'] += model.predict_proba(test_x)[:,1]
res['Attrition'] = res['Attrition']/5

[1]	training's auc: 0.890922	training's binary_logloss: 0.346515	valid_1's auc: 0.744762	valid_1's binary_logloss: 0.456213
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.932492	training's binary_logloss: 0.302351	valid_1's auc: 0.758095	valid_1's binary_logloss: 0.435982
[3]	training's auc: 0.952487	training's binary_logloss: 0.271028	valid_1's auc: 0.733651	valid_1's binary_logloss: 0.439292
[4]	training's auc: 0.97226	training's binary_logloss: 0.239533	valid_1's auc: 0.720635	valid_1's binary_logloss: 0.445689
[5]	training's auc: 0.980559	training's binary_logloss: 0.21559	valid_1's auc: 0.725079	valid_1's binary_logloss: 0.4461
[6]	training's auc: 0.989754	training's binary_logloss: 0.192023	valid_1's auc: 0.74781	valid_1's binary_logloss: 0.439267
[7]	training's auc: 0.991716	training's binary_logloss: 0.174335	valid_1's auc: 0.745778	valid_1's binary_logloss: 0.445503
[8]	training's auc: 0.995299	training's binary_logloss: 0.155654	valid_1's 

[1]	training's auc: 0.869714	training's binary_logloss: 0.367937	valid_1's auc: 0.803518	valid_1's binary_logloss: 0.393266
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.915541	training's binary_logloss: 0.327259	valid_1's auc: 0.811368	valid_1's binary_logloss: 0.376222
[3]	training's auc: 0.93912	training's binary_logloss: 0.293204	valid_1's auc: 0.794082	valid_1's binary_logloss: 0.376375
[4]	training's auc: 0.960132	training's binary_logloss: 0.2653	valid_1's auc: 0.793705	valid_1's binary_logloss: 0.371081
[5]	training's auc: 0.971406	training's binary_logloss: 0.239747	valid_1's auc: 0.806688	valid_1's binary_logloss: 0.359155
[6]	training's auc: 0.981955	training's binary_logloss: 0.21881	valid_1's auc: 0.813708	valid_1's binary_logloss: 0.353643
[7]	training's auc: 0.991889	training's binary_logloss: 0.195764	valid_1's auc: 0.821256	valid_1's binary_logloss: 0.350303
[8]	training's auc: 0.995296	training's binary_logloss: 0.175634	valid_1's

[92]	training's auc: 1	training's binary_logloss: 8.19324e-05	valid_1's auc: 0.879227	valid_1's binary_logloss: 0.571305
[93]	training's auc: 1	training's binary_logloss: 7.61782e-05	valid_1's auc: 0.880133	valid_1's binary_logloss: 0.575969
[94]	training's auc: 1	training's binary_logloss: 7.01188e-05	valid_1's auc: 0.880888	valid_1's binary_logloss: 0.586269
[95]	training's auc: 1	training's binary_logloss: 6.37595e-05	valid_1's auc: 0.882095	valid_1's binary_logloss: 0.589484
[96]	training's auc: 1	training's binary_logloss: 5.85199e-05	valid_1's auc: 0.882095	valid_1's binary_logloss: 0.59428
[97]	training's auc: 1	training's binary_logloss: 5.47046e-05	valid_1's auc: 0.881944	valid_1's binary_logloss: 0.597383
[98]	training's auc: 1	training's binary_logloss: 5.03929e-05	valid_1's auc: 0.883152	valid_1's binary_logloss: 0.59725
[99]	training's auc: 1	training's binary_logloss: 4.69417e-05	valid_1's auc: 0.882095	valid_1's binary_logloss: 0.602067
[100]	training's auc: 1	training's

[65]	training's auc: 1	training's binary_logloss: 0.000729908	valid_1's auc: 0.778802	valid_1's binary_logloss: 0.658446
[66]	training's auc: 1	training's binary_logloss: 0.000669425	valid_1's auc: 0.779826	valid_1's binary_logloss: 0.662852
[67]	training's auc: 1	training's binary_logloss: 0.000607069	valid_1's auc: 0.779485	valid_1's binary_logloss: 0.669467
[68]	training's auc: 1	training's binary_logloss: 0.000545689	valid_1's auc: 0.77846	valid_1's binary_logloss: 0.679926
[69]	training's auc: 1	training's binary_logloss: 0.000502244	valid_1's auc: 0.779143	valid_1's binary_logloss: 0.681868
[70]	training's auc: 1	training's binary_logloss: 0.000456458	valid_1's auc: 0.775047	valid_1's binary_logloss: 0.692155
[71]	training's auc: 1	training's binary_logloss: 0.000413869	valid_1's auc: 0.774364	valid_1's binary_logloss: 0.697916
[72]	training's auc: 1	training's binary_logloss: 0.000380559	valid_1's auc: 0.774023	valid_1's binary_logloss: 0.707035
[73]	training's auc: 1	training's

[44]	training's auc: 1	training's binary_logloss: 0.00567114	valid_1's auc: 0.767193	valid_1's binary_logloss: 0.489228
[45]	training's auc: 1	training's binary_logloss: 0.00523642	valid_1's auc: 0.76807	valid_1's binary_logloss: 0.494288
[46]	training's auc: 1	training's binary_logloss: 0.00477358	valid_1's auc: 0.769474	valid_1's binary_logloss: 0.497888
[47]	training's auc: 1	training's binary_logloss: 0.00435686	valid_1's auc: 0.768772	valid_1's binary_logloss: 0.507389
[48]	training's auc: 1	training's binary_logloss: 0.00396893	valid_1's auc: 0.770702	valid_1's binary_logloss: 0.509408
[49]	training's auc: 1	training's binary_logloss: 0.00367038	valid_1's auc: 0.774386	valid_1's binary_logloss: 0.509374
[50]	training's auc: 1	training's binary_logloss: 0.0033337	valid_1's auc: 0.777719	valid_1's binary_logloss: 0.510164
[51]	training's auc: 1	training's binary_logloss: 0.00304543	valid_1's auc: 0.775789	valid_1's binary_logloss: 0.518111
[52]	training's auc: 1	training's binary_l

[15]	training's auc: 1	training's binary_logloss: 0.0749088	valid_1's auc: 0.710447	valid_1's binary_logloss: 0.428164
[16]	training's auc: 1	training's binary_logloss: 0.0671869	valid_1's auc: 0.711202	valid_1's binary_logloss: 0.43305
[17]	training's auc: 1	training's binary_logloss: 0.0607901	valid_1's auc: 0.714221	valid_1's binary_logloss: 0.441716
[18]	training's auc: 1	training's binary_logloss: 0.0553404	valid_1's auc: 0.713919	valid_1's binary_logloss: 0.445471
[19]	training's auc: 1	training's binary_logloss: 0.0506853	valid_1's auc: 0.720562	valid_1's binary_logloss: 0.446713
[20]	training's auc: 1	training's binary_logloss: 0.0459388	valid_1's auc: 0.721769	valid_1's binary_logloss: 0.452096
[21]	training's auc: 1	training's binary_logloss: 0.0408953	valid_1's auc: 0.718448	valid_1's binary_logloss: 0.460827
[22]	training's auc: 1	training's binary_logloss: 0.0370836	valid_1's auc: 0.723279	valid_1's binary_logloss: 0.4628
[23]	training's auc: 1	training's binary_logloss: 0

In [43]:
valid_pre = model.predict(train_x)
mea_res = mea_res.append(measures(train_y, valid_pre)).reset_index(drop=True)
mea_res

accuracy      : 0.971
precision score: 0.987
recall score  : 0.831
f1 score      : 0.902
roc-auc score      : 0.915


,accuracy,precision,recall,f1,auc
0,0.838182,0.000000,0.000000,0.000000,0.500000
1,0.964545,1.000000,0.780899,0.876972,0.890449
2,0.968182,0.967320,0.831461,0.894260,0.913019
3,0.966364,0.937888,0.848315,0.890855,0.918734
4,0.970909,0.986667,0.831461,0.902439,0.914646


#### baseline

In [28]:
model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=70, reg_alpha=0., reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary', metric='F1',
    subsample=0.95, colsample_bytree=0.95, subsample_frew=1, 
    learning_rate=0.002, random_state=2017)

train_x = data[data['Attrition'] != -1][feats]
train_y = data[data['Attrition'] != -1]['Attrition']
test_x = data[data['Attrition'] == -1][feats]
res = data[data['Attrition'] == -1][['Attrition']]

res['Attrition']=0
for train_idx, val_idx in kfold.split(train_x):
    model.random_state = model.random_state+1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x.loc[val_idx]
    test_y1 = train_y.loc[val_idx]
    model.fit(train_x1, train_y1, eval_set=[(train_x1, train_y1), (test_x1, test_y1)],
        eval_metric='auc', early_stopping_rounds=100)
    res['Attrition'] += model.predict_proba(test_x)[:,1]
res['Attrition'] = res['Attrition']/5

[1]	training's auc: 0.890001	valid_1's auc: 0.752698
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.902683	valid_1's auc: 0.752127
[3]	training's auc: 0.900852	valid_1's auc: 0.751746
[4]	training's auc: 0.902256	valid_1's auc: 0.748444
[5]	training's auc: 0.902291	valid_1's auc: 0.745587
[6]	training's auc: 0.901526	valid_1's auc: 0.742857
[7]	training's auc: 0.900922	valid_1's auc: 0.741333
[8]	training's auc: 0.900338	valid_1's auc: 0.741968
[9]	training's auc: 0.901697	valid_1's auc: 0.743619
[10]	training's auc: 0.90525	valid_1's auc: 0.748571
[11]	training's auc: 0.907817	valid_1's auc: 0.751683
[12]	training's auc: 0.910046	valid_1's auc: 0.759937
[13]	training's auc: 0.911163	valid_1's auc: 0.75854
[14]	training's auc: 0.913529	valid_1's auc: 0.761714
[15]	training's auc: 0.913856	valid_1's auc: 0.767302
[16]	training's auc: 0.914254	valid_1's auc: 0.768698
[17]	training's auc: 0.914686	valid_1's auc: 0.770349
[18]	training's auc: 0.915441	v

[43]	training's auc: 0.90606	valid_1's auc: 0.824577
[44]	training's auc: 0.9067	valid_1's auc: 0.825936
[45]	training's auc: 0.90858	valid_1's auc: 0.827597
[46]	training's auc: 0.90919	valid_1's auc: 0.828502
[47]	training's auc: 0.910436	valid_1's auc: 0.830012
[48]	training's auc: 0.910464	valid_1's auc: 0.829559
[49]	training's auc: 0.911447	valid_1's auc: 0.831975
[50]	training's auc: 0.911409	valid_1's auc: 0.832277
[51]	training's auc: 0.912716	valid_1's auc: 0.833484
[52]	training's auc: 0.913041	valid_1's auc: 0.833333
[53]	training's auc: 0.913174	valid_1's auc: 0.834239
[54]	training's auc: 0.913222	valid_1's auc: 0.8359
[55]	training's auc: 0.913499	valid_1's auc: 0.8359
[56]	training's auc: 0.914873	valid_1's auc: 0.834994
[57]	training's auc: 0.914768	valid_1's auc: 0.835145
[58]	training's auc: 0.914854	valid_1's auc: 0.836051
[59]	training's auc: 0.914968	valid_1's auc: 0.836202
[60]	training's auc: 0.915083	valid_1's auc: 0.836353
[61]	training's auc: 0.915121	valid_1

[233]	training's auc: 0.947679	valid_1's auc: 0.847675
[234]	training's auc: 0.947861	valid_1's auc: 0.848128
[235]	training's auc: 0.948061	valid_1's auc: 0.847826
[236]	training's auc: 0.948109	valid_1's auc: 0.847826
[237]	training's auc: 0.94829	valid_1's auc: 0.847977
[238]	training's auc: 0.948357	valid_1's auc: 0.847826
[239]	training's auc: 0.948452	valid_1's auc: 0.847977
[240]	training's auc: 0.948567	valid_1's auc: 0.847675
[241]	training's auc: 0.948767	valid_1's auc: 0.847524
[242]	training's auc: 0.948863	valid_1's auc: 0.847373
[243]	training's auc: 0.948958	valid_1's auc: 0.847373
[244]	training's auc: 0.949149	valid_1's auc: 0.847222
[245]	training's auc: 0.949292	valid_1's auc: 0.84692
[246]	training's auc: 0.949406	valid_1's auc: 0.847071
[247]	training's auc: 0.949569	valid_1's auc: 0.847071
[248]	training's auc: 0.949626	valid_1's auc: 0.847071
[249]	training's auc: 0.949845	valid_1's auc: 0.847524
[250]	training's auc: 0.949836	valid_1's auc: 0.847675
[251]	traini

[103]	training's auc: 0.922855	valid_1's auc: 0.729903
[104]	training's auc: 0.922827	valid_1's auc: 0.729732
[105]	training's auc: 0.92292	valid_1's auc: 0.729903
[106]	training's auc: 0.922994	valid_1's auc: 0.729561
[107]	training's auc: 0.922827	valid_1's auc: 0.729561
[108]	training's auc: 0.922864	valid_1's auc: 0.729391
[109]	training's auc: 0.923059	valid_1's auc: 0.729732
[110]	training's auc: 0.922882	valid_1's auc: 0.729561
[111]	training's auc: 0.922984	valid_1's auc: 0.728537
[112]	training's auc: 0.923022	valid_1's auc: 0.728708
[113]	training's auc: 0.923244	valid_1's auc: 0.728196
[114]	training's auc: 0.923337	valid_1's auc: 0.728537
[115]	training's auc: 0.923458	valid_1's auc: 0.72922
[116]	training's auc: 0.923792	valid_1's auc: 0.730244
[117]	training's auc: 0.924145	valid_1's auc: 0.730073
[118]	training's auc: 0.92421	valid_1's auc: 0.729391
[119]	training's auc: 0.924302	valid_1's auc: 0.729561
[120]	training's auc: 0.924339	valid_1's auc: 0.729561
[121]	trainin

[110]	training's auc: 0.93077	valid_1's auc: 0.752114
[111]	training's auc: 0.931486	valid_1's auc: 0.751661
[112]	training's auc: 0.931772	valid_1's auc: 0.752566
[113]	training's auc: 0.932001	valid_1's auc: 0.753321
[114]	training's auc: 0.932202	valid_1's auc: 0.753623
[115]	training's auc: 0.932297	valid_1's auc: 0.754227
[116]	training's auc: 0.932412	valid_1's auc: 0.754831
[117]	training's auc: 0.932526	valid_1's auc: 0.75468
[118]	training's auc: 0.932602	valid_1's auc: 0.754982
[119]	training's auc: 0.93265	valid_1's auc: 0.754831
[120]	training's auc: 0.932602	valid_1's auc: 0.755284
[121]	training's auc: 0.932765	valid_1's auc: 0.756492
[122]	training's auc: 0.93287	valid_1's auc: 0.756793
[123]	training's auc: 0.93286	valid_1's auc: 0.756341
[124]	training's auc: 0.93308	valid_1's auc: 0.758001
[125]	training's auc: 0.93328	valid_1's auc: 0.757699
[126]	training's auc: 0.933366	valid_1's auc: 0.75785
[127]	training's auc: 0.93348	valid_1's auc: 0.757699
[128]	training's au

In [27]:
res.shape, test_x.shape

((350, 1), (350, 30))